In [0]:
%sql
use catalog dev;
use schema vansh_silver

In [0]:
CREATE OR REPLACE TABLE dev.vansh_silver.heartrate_device (device_id INT, mrn STRING, name STRING, time TIMESTAMP, heartrate DOUBLE);

INSERT INTO dev.vansh_silver.heartrate_device VALUES
  (23, "40580129", "Nicholas Spears", "2020-02-01T00:01:58.000+0000", 54.0122153343),
  (17, "52804177", "Lynn Russell", "2020-02-01T00:02:55.000+0000", 92.5136468131),
  (37, "65300842", "Samuel Hughes", "2020-02-01T00:08:58.000+0000", 52.1354807863),
  (23, "40580129", "Nicholas Spears", "2020-02-01T00:16:51.000+0000", 54.6477014191),
  (17, "52804177", "Lynn Russell", "2020-02-01T00:18:08.000+0000", 95.033344842);
  
SELECT * FROM dev.vansh_silver.heartrate_device

In [0]:
CREATE OR REPLACE VIEW dev.vansh_gold.agg_heartrate AS (
  SELECT mrn, name, MEAN(heartrate) avg_heartrate, DATE_TRUNC("DD", time) date
  FROM dev.vansh_silver.heartrate_device
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);


In [0]:
SELECT * FROM dev.vansh_gold.agg_heartrate

In [0]:
CREATE OR REPLACE VIEW dev.vansh_gold.agg_heartrate AS (
  SELECT 
  CASE WHEN
    is_account_group_member('bi') THEN 'REDACTED'
    ELSE mrn
  END AS mrn, 
  name, 
  MEAN(heartrate) avg_heartrate, 
  DATE_TRUNC("DD", time) date
  FROM dev.vansh_silver.heartrate_device
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);

In [0]:
select * from dev.vansh_gold.agg_heartrate

In [0]:
create or replace view dev.vansh_gold.heartrate 
as 
SELECT * FROM heartrate_device  
where
CASE
    WHEN is_account_group_member('de') THEN device_id<30
    ELSE True
  END;

In [0]:
select * from vansh_gold.heartrate

In [0]:
CREATE OR REPLACE FUNCTION vansh_gold.datamask(x STRING)
  RETURNS STRING
  RETURN CONCAT(REPEAT("*", LENGTH(x) - 2), RIGHT(x, 2)
); 

In [0]:
select vansh_gold.datamask('naval@gmail.com') as email

In [0]:
CREATE OR REPLACE VIEW vansh_gold.agg_heartrate AS (
  SELECT 
  CASE WHEN
    is_account_group_member('account users') THEN vansh_gold.datamask(mrn)
    ELSE mrn
  END AS mrn, 
  name, 
  MEAN(heartrate) avg_heartrate, 
  DATE_TRUNC("DD", time) date
  FROM heartrate_device
  GROUP BY mrn, name, DATE_TRUNC("DD", time)
);

In [0]:
select * from vansh_gold.agg_heartrate